# E-76 Obligated Federal Funds Data Exploration
### From the Division of Local Assistance 

E-76's or "Authorizations to Proceed" is a federal authorization of funds to establish the reimbursement date for a phase of work 

#### Data Sources: 
- https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated

In this notebook, we will be looking to see what types of data is in the spreadsheet as well as begin to clean it for futher analysis


In [2]:
##one time load 
#!pip install openpyxl


In [1]:
from siuba import _, mutate, count, filter, group_by, ungroup, separate
import numpy as np
import pandas as pd

In [2]:
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)

## Reading Excel as a data dictionary
Can use this approach, but recommend the next with second one. 

In [3]:
#obligated = pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=['2014-2018', '2019', '2020', '2021'])

In [3]:
#print(obligated)

In [4]:
#print(type(obligated))

In [5]:
#len(obligated)

In [6]:
#print(type(obligated['2020']))

In [7]:
#obligated['2020'].info()

In [8]:
#obligated['2021'].info()

In [9]:
#obligated['2019'].info()

In [10]:
#obligated['2014-2018'].info()

In [11]:
#df1 = obligated['2014-2018']
#df2 = obligated['2019']
#df3 = obligated['2020']
#df4 = obligated['2021']

In [12]:
#pd_dct = {"pre2018": df1, "2019": df2, "2020":df3, "2021":df4}

In [13]:
#mdf = pd.concat(pd_dct, axis=0).reset_index(level=0).rename({'level_0':'key'}, axis=1)

In [14]:
#mdf

## Reading Excel with Concat

In [4]:
#attempt 2
## code source: https://pbpython.com/pandas-excel-tabs.html
odf = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/obligated-projects.xlsx', sheet_name=None), ignore_index=True)



/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
odf.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN


In [6]:
odf.tail()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
20016,Obligated,BRLO,5955(095),Orange County,2021-08-20 00:00:00,2021-08-27 00:00:00,2021-08-31 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,82000.00,0.00,82000.00,Authorized,5955,12,E-76 approved on,NaN,7.00,5.00,16.00,ORA020501,"Silverado Canyon Road Over Silverado Creek, Ap...",Bridge Replacement (tc).,2,2021-08-20 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20017,Obligated,HSIPL,5073(093),Orange,2021-08-17 00:00:00,2021-08-27 00:00:00,2021-08-30 00:00:00,2021-08-30 00:00:00,2021-09-08 00:00:00,21000.00,0.00,21000.00,Authorized,5073,12,E-76 approved on,NaN,30.00,3.00,9.00,SCAG015,Intersection Of Glassell St. & Palmyra Ave.,Install A New 2-phase Traffic Signal,2,2021-07-28 00:00:00,2021-08-26 00:00:00,SCAG,NaT,NaN
20018,Obligated,HSIPL,5073(078),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.00,0.00,188921.73,Authorized,5073,12,E-76 approved on,NaN,14.00,5.00,16.00,SCAG015,Intersection Of Glassell Street And Walnut Avenue,Protected Left-turn Signal Phasing,4,2021-08-13 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20019,Obligated,HSIPL,5073(080),Orange,2021-08-18 00:00:00,2021-08-27 00:00:00,2021-09-01 00:00:00,2021-09-01 00:00:00,2021-09-17 00:00:00,0.00,0.00,69358.21,Authorized,5073,12,E-76 approved on,NaN,15.00,5.00,16.00,SCAG015,Intersection Of Chapman Avenue And James Street,Install Protected Left Turn Signal Phasing,4,2021-08-12 00:00:00,2021-08-27 00:00:00,SCAG,NaT,NaN
20020,Obligated,CMSTPLN,6071(129),Orange County Transportation Authority,2021-08-23 00:00:00,2021-08-24 00:00:00,2021-08-31 00:00:00,2021-08-31 00:00:00,2021-09-09 00:00:00,8000000.00,47628875.00,53215649.00,Authorized,6071,12,E-76 approved on,NaN,84.00,7.00,9.00,ORA100511,State Route 55 From Interstate 405 To Intersta...,Add One Mixed-flow And High-occupancy Lane In ...,6,2021-06-01 00:00:00,2021-08-22 00:00:00,SCAG,NaT,NaN


In [7]:
print(len(odf))

20021


In [8]:
odf.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28
19191,Obligated,STPL,5241(073),Ceres,2021-03-09 00:00:00,2021-03-09 00:00:00,2021-04-14 00:00:00,2021-04-30 00:00:00,2021-05-03 00:00:00,527673.00,0.00,871262.00,Authorized,5241,10,E-76 approved on,NaN,4.00,52.00,3.00,VA02,Blaker Rd - Hackett Rd To Whitmore Ave Central...,Asphalt Concrete Overlay,1,2021-03-05 00:00:00,2021-03-08 00:00:00,STANCOG,NaT,NaN
14072,Obligated,CMSTPLN,6053(130),San Bernardino Associated Governments,2019-05-08 00:00:00,2019-05-22 00:00:00,2019-06-05 00:00:00,2019-06-06 00:00:00,2019-07-02 00:00:00,6862056.00,-6862056.00,0,Authorized,6053,8,E-76 approved on,NaN,22.00,15.00,26.00,20159902,I-10 Corridor Express Lane Widening From San A...,Construct Toll Express Lanes (tc),6,2019-04-30 00:00:00,2019-05-21 00:00:00,SCAG,NaN,NaN
17161,Obligated,HSIPL,5927(112),Marin County,2020-06-26 00:00:00,2020-06-26 00:00:00,2020-07-02 00:00:00,2020-08-31 00:00:00,2020-09-09 00:00:00,1009500.00,0.00,1975900.00,Authorized,5927,4,E-76 approved on,NaN,2.00,66.00,9.00,VAR170002,Lucas Valley Road Mp 5.08,"Improve Curve Alignment Of Lucas Valley Road, ...",2,2020-06-24 00:00:00,2020-06-26 00:00:00,MTC,NaN,NaN
5205,Obligated,CMLNI,5115(032),Lemoore,2016-12-01,2016-12-05,2016-12-07,2016-12-09,2016-12-23,120000.00,0.00,514496.00,Authorized,5115.00,6,E-76 approved on Prepared on 10/6/2015,NaN,4.00,4.00,14.00,CM5115015,City Of Lemoore,Purchase One Cng Truck,1,NaN,NaN,KCAG,NaN,NaN
16285,Obligated,BRLS,5203(042),Chula Vista,2020-02-11 00:00:00,2020-02-14 00:00:00,2020-02-18 00:00:00,2020-02-18 00:00:00,2020-03-04 00:00:00,177060.00,0.00,200000.00,Authorized,5203,11,E-76 approved on,NaN,30.00,4.00,15.00,CHV08,Willow Street Bridge At Sweetwater River. New ...,Years 2 To 5 Environmental Compliance (mitigat...,1,2020-01-15 00:00:00,2020-02-11 00:00:00,SDAG,NaN,NaN
69,Obligated,VPPL,6073(030),University Of California,2018-10-16,2018-10-16,2018-12-13,2018-12-13,2018-12-14,-19758.57,0.00,-24708.61,Authorized,6073.00,4,E-76 approved on,NaN,NaN,58.00,1.00,ALA150029,Within City Of Berkeley,Study On-campus Parking Pricing,2,NaN,NaN,NON-MPO,NaN,NaN
4480,Obligated,HSIPL,5466(023),Yucca Valley,2017-04-05,2017-04-06,2017-04-12,2017-04-24,2017-05-01,139500.00,0.00,155000.00,Authorized,5466.00,8,E-76 approved on Prepared on 2/14/2017,NaN,1.00,18.00,7.00,SCAG015,Yucca Trail From The Intersection Of Warren Vi...,Widen Roadway To Construct Left-turn Lane,1,NaN,NaN,SCAG,NaN,NaN
14153,Obligated,BRLS,5938(199),Stanislaus County,2019-04-25 00:00:00,2019-05-03 00:00:00,2019-07-09 00:00:00,2019-07-09 00:00:00,2019-07-16 00:00:00,18529329.00,0.00,22080000,Authorized,5938,10,E-76 approved on,NaN,35.00,67.00,7.00,STANCOG,Hickman Road (bridge 38c0004) Over Tuolumne River,Replace 2-lane Bridge With A Wider Bridge,3,2019-03-29 00:00:00,2019-04-29 00:00:00,STANCOG,NaN,NaN
4312,Obligated,BRLS,5924(155),Sacramento County,2017-05-11,2017-05-11,2017-05-18,2017-05-23,2017-05-30,362973.00,0.00,410000.00,Authorized,5924.00,3,E-76 approved on Prepared on 2/14/2017,NaN,0.00,12.00,7.00,NaN,Franklin Blvd At Lost Slough Br# 24c0155,Bridge Replacement,2,NaN,NaN,SACOG,NaN,NaN
17651,Obligated,CML,5254(020),Atwater,2020-11-03 00:00:00,2020-11-03 00:00:00,2020-11-03 00:00:00,2020-11-03 00:00:00,2020-11-10 00:00:00,0.00,0.00,2013.49,Authorized,5254,10,E-76 approved on,NaN,5.00,0.00,7.00,CMAQ14-12,Buhach Rd - Avenue One To Buhach Colony High S...,Sidewalk Infill,3,2020-10-29 00:00:00,2020-10-30 00:00:00,MCAG,NaN,NaN


In [9]:
## Adding Waiting data too

In [10]:
wdf = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/E-76-Waiting-list.xlsx', sheet_name=None), ignore_index=True)



In [11]:
wdf.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Warning
0,DISTRICT,BPMP,5904(141),Humboldt County,2021-08-11,NaT,NaT,NaT,NaT,53348.54,0.00,60260.00,Hold for FTIP amendment.,5904,1,Prepared on 8/11/2021,43039.00,NaN,NaN,NaN,NaN,In Humboldt County On Mccann Road Over The Eel...,Bridge Preventive Maintenance,5,NaT,NaT,NON-MPO,2021-11-01,NaN
1,DISTRICT,ATPL,5904(151),Humboldt County,2021-10-22,NaT,NaT,NaT,NaT,1358000.00,0.00,1358000.00,..,5904,1,Prepared on 10/22/2021,10.00,NaN,NaN,NaN,NaN,On Sr255 From The Dean Avenue/pacific Avenue I...,Class 1 Multi-use Trail (tc),5,NaT,NaT,NON-MPO,NaT,NaN
2,DISTRICT,BPMP,5904(156),Humboldt County,2021-08-05,NaT,NaT,NaT,NaT,629249.10,0.00,710775.00,Hold Obligation authority or apportionment bal...,5904,1,Prepared on 8/5/2021,88.00,NaN,NaN,NaN,NaN,Various Locations In Humboldt County,Bridge Preventive Maintenance: Minor Concrete ...,3,NaT,NaT,NON-MPO,NaT,NaN
3,DISTRICT,BRLO,5904(165),Humboldt County,2021-09-13,NaT,NaT,NaT,NaT,9607.61,0.00,9607.61,..,5904,1,Prepared on 9/13/2021,49.00,NaN,NaN,NaN,NaN,In Humboldt County On Jacoby Creek Bridge At B...,Bridge Rehabilitation Child Project Revegetati...,2,NaT,NaT,NON-MPO,NaT,NaN
4,DISTRICT,HSIPL,5904(181),Humboldt County,2021-10-20,NaT,NaT,NaT,NaT,154662.00,0.00,178643.00,Hold for FTIP amendment.,5904,1,Prepared on 10/20/2021,12.00,NaN,NaN,NaN,NaN,Intersection Of Redwood Drive And Redway Drive...,Install Enhanced Pedestrian Crossing With Bulb...,3,NaT,NaT,NON-MPO,NaT,NaN


In [12]:
df = pd.concat([odf, wdf], ignore_index=True)

In [13]:
df.head()

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28,Warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN,NaN


In [14]:
df.sample(10)

,Location,PREFIX,Project NO,Agency,prepared date,submit to HQ date,HQ review date,submit to FHWA date,To FMIS date,Fed requested,AC requested,total requested,Status Comment,Locode,DIST,Status,Waiting Days,dist processing Days,HQ processing Days,FHWA processing Days,FTIP No,project location,Type of Work,SEQ,Date Request Initiated,date completed request,MPO,Today,Unnamed: 28,Warning
9430,Obligated,CML,5393(031),Camarillo,2015-04-17,2015-04-17,2015-04-20,2015-04-20,2015-04-22,0.00,0.00,114729.00,Authorized,5393.00,7,E-76 approved on E-76 approved on,NaN,0.00,3.00,2.00,VEN110105,In City Of Camarillo From Hwy 101 To Pleasant ...,Construction Of Bike Trail Phase 3 Along Calle...,2,NaN,NaN,SCAG,NaN,NaN,NaN
19480,Obligated,BRLS,5929(238),San Joaquin County,2021-02-03 00:00:00,2021-02-05 00:00:00,2021-06-02 00:00:00,2021-06-10 00:00:00,2021-06-15 00:00:00,86405.00,0.00,97600.00,Authorized,5929,10,E-76 approved on,NaN,56.00,125.00,5.00,SJ07-3002,Escalon Bellota Road Over Little Johns Creek (...,Bridge Replacement,5,2020-12-11 00:00:00,2021-02-03 00:00:00,SJCG,NaT,NaN,NaN
15355,Obligated,HSIPL,5063(155),Santa Ana,2019-11-06 00:00:00,2019-11-06 00:00:00,2019-11-06 00:00:00,2019-11-06 00:00:00,2019-11-19 00:00:00,-4996.84,0.00,88877.48,Authorized,5063,12,E-76 approved on,NaN,6.00,0.00,13.00,SCAG015,Various Locations Throughout The City Of Santa...,Install Crosswalks And Flashing Beacons (tc),3,2019-10-31 00:00:00,2019-11-01 00:00:00,SCAG,NaN,NaN,NaN
16915,Obligated,BHLO,5435(017),Agoura Hills,2019-08-06 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-17 00:00:00,2840400.00,0.00,2920400.01,Authorized,5435,7,E-76 approved on,NaN,82.00,0.00,10.00,LA000800,"Roadside Drive Over Medea Creek, 0.3 Mile East...",Bridge Rehabilitation. (tc),5,2020-04-16 00:00:00,2020-06-17 00:00:00,SCAG,NaN,NaN,NaN
10403,Obligated,HSIPL,5068(044),Redding,2014-11-25,2014-11-25,2014-12-01,2014-12-01,2014-12-02,99766.27,0.00,156792.84,Authorized,5068.00,2,E-76 approved on E-76 approved on,NaN,0.00,6.00,1.00,GP-HSIP,Lake Boulevard From State Route 299 To Northpo...,"Raised Median, Striping, Markings, Lane Reconfig.",3,NaN,NaN,STPA,NaN,NaN,NaN
11979,Obligated,STPL,5404(025),Lafayette,2013-10-17,2014-04-29,2014-04-30,2014-05-01,2014-05-02,584000.00,0.00,689000.00,Authorized,5404.00,4,E-76 approved on E-76 approved on,NaN,194.00,2.00,1.00,CC-130009,Mt. Diablo Blvd. From Mt. View Drive To El Nid...,Ped/bike Walkway Improvement,1,NaN,NaN,MTC,NaN,NaN,NaN
14984,Obligated,CML,5005(127),San Jose,2019-10-02 00:00:00,2019-10-02 00:00:00,2019-10-02 00:00:00,2019-10-02 00:00:00,2019-10-10 00:00:00,-46.13,0.00,6917.55,Authorized,5005,4,E-76 approved on,NaN,13.00,0.00,8.00,SCL130010,In San Jose At Various Intersections - See Sta...,Implement Traffic Signal Controlled Crossings,5,2019-09-19 00:00:00,2019-09-20 00:00:00,MTC,NaN,NaN,NaN
12543,Obligated,CML,5472(018),Windsor,2014-03-06,2014-03-06,2014-03-10,2014-03-10,2014-03-12,108000.00,0.00,122000.00,Authorized,5472.00,4,E-76 approved on E-76 approved on,NaN,0.00,4.00,2.00,SON130003,Intersection Of Jaguar Way And Windsor Road,"Install Traffic Signal, Construct 800' Bikelanes",1,NaN,NaN,MTC,NaN,NaN,NaN
9822,Obligated,HSIPL,5449(025),Highland,2015-01-12,2015-01-29,2015-02-19,2015-02-19,2015-03-01,0.00,0.00,133.00,Authorized,NaN,8,E-76 approved on E-76 approved on,NaN,17.00,21.00,10.00,SCAG015,9th Street From Del Rosa Drive To Palm Avenue,"Restriping, Flashing Beacons, Lighted Crosswalks",3,NaN,NaN,SCAG,NaN,NaN,NaN
430,Obligated,BRLO,5907(032),Lassen County,2018-10-03,2018-10-03,2018-10-03,2018-10-10,2018-10-16,-4.30,0.00,-5.33,Authorized,5907.00,2,E-76 approved on,NaN,NaN,7.00,6.00,HBPLOCAL,Susan River Overfl Br #5 On Mapes Ln (br. #07c...,Bridge Replacement (tc),9,NaN,NaN,NON-MPO,NaN,NaN,NaN


In [15]:
print(len(df))

20447


## Data Cleaning

In [16]:
from calitp import to_snakecase


In [17]:
df = to_snakecase(df)

In [18]:
df.head(1)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,waiting_days,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,today,unnamed:_28,warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN,NaN


In [20]:
df.agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

Some Agencies are all caps, others are not. Applying str.title() to correct this. 

In [21]:
df.agency = df.agency.str.title()

In [22]:
df.agency.unique()

array(['Humboldt County', 'Mendocino County', 'Sacramento County',
       'Sutter County', 'Contra Costa County', 'Dinuba', 'Arvin',
       'Fresno', 'Tulare County', 'Palmdale', 'Los Angeles County',
       'Orange County Transportation Authority', 'Yolo County',
       'Nevada County', 'Fresno County', 'Ridgecrest', 'Lassen County',
       'Modoc County', 'Butte County', 'Yuba City', 'Concord',
       'Napa County', 'Pismo Beach', 'Ojai', 'Los Angeles', 'Chino Hills',
       'Indio', 'Moreno Valley', 'Turlock', 'Riverbank', 'Stockton',
       'Chula Vista', 'Sonoma County', 'Santa Clara County',
       'Kings County', 'Long Beach', 'Norwalk', 'Rancho Palos Verdes',
       'Southern California Association Of Governments', 'Pasadena',
       'Riverside', 'Carlsbad', 'Santa Barbara County', 'Delano',
       'Merced County', 'South Lake Tahoe', 'Yuba County',
       'Metropolitan Transportation Commission', 'San Mateo',
       'Solano Transportation Authority', 'University Of California'

In [23]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,waiting_days,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,today,unnamed:_28,warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,NaN,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,NaN,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,NaN,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,NaN,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,NaN,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,NaN,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,NaN,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,NaN,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,NaN,NaN


In [24]:
df['waiting_days'].unique()

array([       nan, 4.3039e+04, 1.0000e+01, 8.8000e+01, 4.9000e+01,
       1.2000e+01, 8.0000e+01, 1.1600e+03, 1.2500e+02, 1.2900e+02,
       1.2400e+02, 6.5400e+02, 1.1500e+02, 5.4000e+01, 1.1800e+02,
       1.7900e+02, 3.4700e+02, 6.3600e+02, 5.5000e+02, 1.1000e+01,
       7.3000e+01, 5.0100e+02, 5.7300e+02, 3.3600e+02, 2.4000e+01,
       7.3300e+02, 3.3500e+02, 6.0000e+00, 6.9000e+01, 2.0800e+02,
       2.5500e+02, 1.5590e+03, 2.8000e+01, 6.1000e+01, 5.9000e+01,
       3.5500e+02, 1.5100e+02, 3.9200e+02, 9.6000e+01, 1.7000e+01,
       5.5000e+01, 1.9600e+02, 1.3000e+02, 4.0600e+02, 4.0500e+02,
       4.0400e+02, 5.8100e+02, 2.1000e+01, 9.1000e+01, 1.3900e+02,
       7.4000e+01, 1.4000e+01, 8.1000e+01, 1.1600e+02, 6.8000e+01,
       6.7000e+01, 1.5800e+02, 1.6000e+02, 5.0000e+00, 1.6100e+02,
       1.4300e+02, 1.0360e+03, 1.5930e+03, 5.2000e+01, 1.5880e+03,
       1.0060e+03, 1.2300e+02, 9.8000e+02, 8.2000e+01, 4.4100e+02,
       2.0420e+03, 7.5000e+01, 4.9500e+02, 5.7900e+02, 7.3900e

In [25]:
df.drop('waiting_days', axis=1, inplace=True)

In [26]:
df['today'].unique()

array([nan, '2020-01-01', '2019-06-06 9:00 am', '2021-01-04',
       Timestamp('2021-10-01 00:00:00'), NaT,
       Timestamp('2021-11-01 00:00:00')], dtype=object)

In [27]:
df.drop('today', axis=1, inplace=True)

In [28]:
df.drop('unnamed:_28', axis=1, inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20447 entries, 0 to 20446
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   location                20447 non-null  object 
 1   prefix                  20447 non-null  object 
 2   project_no              20447 non-null  object 
 3   agency                  20447 non-null  object 
 4   prepared_date           20429 non-null  object 
 5   submit__to_hq_date      20272 non-null  object 
 6   hq_review_date          20240 non-null  object 
 7   submit_to_fhwa_date     20222 non-null  object 
 8   to_fmis_date            20222 non-null  object 
 9   fed_requested           20446 non-null  float64
 10  ac_requested            20447 non-null  float64
 11  total_requested         20446 non-null  object 
 12  status_comment          20447 non-null  object 
 13  locode                  18948 non-null  object 
 14  dist                    20447 non-null

### Finding NA in strings 

In [33]:
#converting the objects in total_requested to float 
df['total_requested'] = df['total_requested'].astype(float)

ValueError: could not convert string to float: '2748.3NA999'

Issue here, and in the following row where 'NA' appears between numbers. Need to determine whether or not to manually override in the exel sheet. 

In [31]:
df >> filter(_.total_requested.str.contains("NA"))

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning
15547,Obligated,ER,38Y0(001),Los Angeles County,2019-11-12 00:00:00,2019-12-04 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-19 00:00:00,1017276.52,-1019159.54,2748.3NA999,Authorized,5953,7,E-76 approved on,28.00,2.00,13.00,NaN,"Mulholland Hwy, Et Al Guardrail Replacement","Install Temporary Traffic Signs, Remove And Re...",2,2019-11-06 00:00:00,2019-11-13 00:00:00,SCAG,NaN
15548,Obligated,ER,38Y0(003),Los Angeles County,2019-11-13 00:00:00,2019-11-14 00:00:00,2019-12-05 00:00:00,2019-12-06 00:00:00,2019-12-20 00:00:00,1033518.07,-1001297.90,30169.98NA99,Authorized,5953,7,E-76 approved on,8.00,22.00,14.00,NaN,Kanan Dume Rd From Mm 8.15 To Mm 11.75,"Install Temporary Traffic Signs, Remove & Repl...",2,2019-11-06 00:00:00,2019-11-13 00:00:00,SCAG,NaN


In [1]:
## removing row with string until we can clarify the true entry.

In [34]:
delete_row = df[df["total_requested"]== '2748.3NA999'].index
df = df.drop(delete_row)

In [35]:
delete_row = df[df["total_requested"]== '30169.98NA99'].index
df = df.drop(delete_row)

In [32]:
## another way to identify the entires that contain string
#df.loc[df['total_requested']=='2748.3NA999']
#df.loc[df['total_requested']=='30169.98NA99']

In [36]:
df['total_requested'] = df['total_requested'].astype(float)

In [37]:
print(df['prepared_date'].dtypes)

object


In [38]:
df['prepared_date'].unique()

array(['2018-12-18', '2018-12-17', '2018-12-07', ...,
       Timestamp('2021-10-06 00:00:00'), Timestamp('2021-09-28 00:00:00'),
       Timestamp('2021-09-21 00:00:00')], dtype=object)

In [39]:
print(df.dtypes)

location                   object
prefix                     object
project_no                 object
agency                     object
prepared_date              object
submit__to_hq_date         object
hq_review_date             object
submit_to_fhwa_date        object
to_fmis_date               object
fed_requested             float64
ac_requested              float64
total_requested           float64
status_comment             object
locode                     object
dist                        int64
status                     object
dist_processing_days      float64
hq_processing_days        float64
fhwa_processing_days      float64
ftip_no                    object
project_location           object
type_of_work               object
seq                         int64
date_request_initiated     object
date_completed_request     object
mpo                        object
warning                    object
dtype: object


In [41]:
cols = ['prepared_date','to_fmis_date','submit_to_fhwa_date','submit__to_hq_date','hq_review_date','date_request_initiated','date_completed_request']
df[cols] = df[cols].applymap(lambda x : pd.to_datetime(x, format = '%Y-%m-%d'))


In [42]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit__to_hq_date      20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

## Separate Project Number into two columns

In [46]:
df[["projectID", "projectNO"]] = df["project_no"].str.split(pat="(", expand=True)


In [48]:
df.projectNO = [x.replace(")", "") for x in df.projectNO]

In [49]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904.00,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN,5904,121
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910.00,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904.00,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924.00,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924.00,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214


## Locode and Project ID Columns 

Hypothesis:
    Project IDs and Locodes, or unique agency identifiers, are the same 
    
As it stands now, these columns have a mix of letters and numbers, making it hard to compare the two. 
In this block we will work to convert these columns, while trying to not convert the non-numeric numbers. 

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit__to_hq_date      20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [51]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO
10485,Obligated,DEM05L,5063(145),Santa Ana,2013-07-30,2014-10-28,2014-11-14,2014-11-14,2014-11-18,3909380.00,0.00,18936000.00,Authorized,NaN,12,E-76 approved on E-76 approved on,455.00,17.00,4.00,ORA125,Bristol Street From Warner Avenue To St. Andre...,Roadway Widening,2,NaT,NaT,SCAG,NaN,5063,145
16169,Obligated,STPHIPL,5109(249),Bakersfield,2020-02-20,2020-02-24,2020-02-24,2020-02-24,2020-03-05,495053.59,0.00,559194.18,Authorized,5109,6,E-76 approved on,13.00,0.00,10.00,KER180403,District Blvd; Between Gosford Rd To Stine Rd ...,Road Reconstruction,2,2020-02-11,2020-02-20,KCOG,NaN,5109,249
3341,Obligated,ATPL,5438(017),East Palo Alto,2017-10-04,2017-10-10,2017-10-10,2017-10-11,2017-10-18,0.00,0.00,1936551.00,Authorized,5438.00,4,E-76 approved on,6.00,1.00,7.00,SM-130028,Us101 At Clarke Avenue/newell Road,Pedestrian/bicycle Overcrossing,2,2017-10-04,2017-10-04,MTC,NaN,5438,017
12038,Obligated,HSIPL,5129(082),Oxnard,2014-04-14,2014-04-14,2014-04-15,2014-04-24,2014-04-25,149400.00,0.00,231665.00,Authorized,5129.00,7,E-76 approved on E-76 approved on,0.00,10.00,1.00,SCAG015,The Intersection Of Rice Avenue And Latigo Avenue,Modify Existing Traffic Signal To Provide Lt Turn,2,NaT,NaT,SCAG,NaN,5129,082
12975,Obligated,SRTSL,5927(061),Marin County,2013-10-22,2013-10-22,2013-12-09,2014-01-07,2014-01-07,350460.37,0.00,350460.37,Authorized,5927.00,4,E-76 approved on E-76 approved on,0.00,77.00,0.00,VAR991010,Marin Ave. & Bell Lane Btw Green Glen Way;sch,Srts Fed Funding Safety C&g; S,5,NaT,NaT,MTC,NaN,5927,061
8814,Obligated,CML,5165(080),Turlock,2015-06-19,2015-06-22,2015-07-02,2015-07-03,2015-07-07,540637.00,0.00,540637.00,Authorized,NaN,10,E-76 approved on E-76 approved on,3.00,11.00,4.00,VA01,Golden State Blvd And Fulkerth Rd Intersection,Intersection Improvements (tc),2,NaT,NaT,SCAG,NaN,5165,080
1952,Obligated,STPL,5059(132),Modesto,2018-04-09,2018-04-09,2018-04-09,2018-04-09,2018-04-14,0.00,0.00,0.00,Authorized,5059.00,10,E-76 approved on,NaN,0.00,5.00,NaN,Stanislaus County And Select Cities In County,Develop Pavement Management Sy,4,NaT,NaT,STANCOG,NaN,5059,132
13828,Obligated,CML,5224(015),Firebaugh,2019-04-23,2019-04-23,2019-04-23,2019-04-23,2019-04-30,0.00,0.00,0.00,Authorized,5224,6,E-76 approved on,5.00,0.00,7.00,FRE110140,On Q Street From 9th & Q St To Canal & Fr Crai...,Pedestrian/bike Lane/ped Bridge,5,2019-04-18,2019-04-19,CFCG,NaN,5224,015
9256,Obligated,STPLZ,6272(032),San Francisco County Transportation Authority,2015-04-20,2015-04-20,2015-05-12,2015-05-12,2015-05-15,26520.00,0.00,29956.00,Authorized,6272.00,4,E-76 approved on E-76 approved on,0.00,22.00,3.00,SF-070027,"Treasure Island Rd, Viaduct,0.3km W Of Bay Br,...",Seismic Analysis And Retrofit Strategy,4,NaT,NaT,MTC,NaN,6272,032
2504,FTA Transferred,FTACML,6155(098),Ventura County Transportation Commission,2018-01-29,2018-01-29,2018-01-29,2018-02-12,2018-02-20,2492859.00,0.00,2492859.00,Prog Code Z400,6155.00,7,FTA transferred on 2/20/2018,NaN,14.00,8.00,NaN,NaN,FTA Transfer,1,NaT,NaT,SCAG,NaN,6155,098


In [51]:
## dataset has a column NBIL, 32L0, 40A0, 5.92E+03, 38Y0, 38R0

## for now, leaving the errors. 

In [52]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

        
# https://stackoverflow.com/questions/56474599/how-to-ignore-errors-in-pandas-to-numeric-and-change-str-to-int

In [53]:
df['locode'] = df['locode'].apply(get_num)

In [54]:
df['projectID'] = df['projectID'].apply(get_num)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit__to_hq_date      20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [56]:
compare_col = np.where(df["locode"] == df["projectID"], True, False)
df["compare_id_locode"] = compare_col
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.00,0.00,Authorized,5904,1,E-76 approved on,NaN,0.00,9.00,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.00,247002.00,Authorized,5924,3,E-76 approved on,4.00,16.00,0.00,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.00,5702041.00,Authorized,5924,3,E-76 approved on,7.00,16.00,0.00,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [57]:
df['compare_id_locode'].value_counts()

True     16677
False     3768
Name: compare_id_locode, dtype: int64

## Getting Unique Locodes

In [61]:
group = df.groupby('agency')
crosswalk_all = group.apply(lambda x: x['locode'].unique())
print(crosswalk_all)

agency
Access Services                                             [6312]
Agoura Hills                                           [5435, nan]
Ala-Con Costa T                                             [6002]
Alameda                                                     [5014]
Alameda - Contra Costa Transit District                     [6002]
                                                    ...           
Yrts                                                         [nan]
Yuba City                                                   [5163]
Yuba County                                                 [5916]
Yucaipa                                    [5457, nan, 5954, NBIL]
Yucca Valley                                           [5466, nan]
Length: 671, dtype: object


In [62]:
#crosswalk_all.to_csv('crosswalk_all.csv')

In [63]:
#https://stackoverflow.com/questions/29530232/how-to-check-if-any-value-is-nan-in-a-pandas-dataframe

nan_rows = df[df['locode'].isnull()]

In [64]:
nan_rows.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
8039,Obligated,STPL,5255(048),Riverbank,2015-11-10,2015-11-10,2015-11-10,2015-11-10,2015-11-13,0.00,0.00,0.00,Authorized,NaN,10,E-76 approved on E-76 approved on,0.00,0.00,3.00,VA02,Claus Road (california To Townsend),Ac Overlay Rehabilitation/resurfacing (tc),5,NaT,NaT,MCTC,NaN,5255,048,False
10825,Obligated,RPSTPL,5410(073),Irvine,2014-09-10,2014-09-10,2014-09-11,2014-09-11,2014-09-12,0.00,0.00,-54601.00,Authorized,NaN,12,E-76 approved on E-76 approved on,0.00,1.00,1.00,ORA990906,Campus Drive From California Avenue To Culver ...,Class I Bikeway,2,NaT,NaT,KCOG,NaN,5410,073,False
11505,Obligated,HSIPL,5441(046),Moreno Valley,2014-04-29,2014-06-16,2014-06-30,2014-06-30,2014-07-07,-164683.00,0.00,-366666.00,Authorized,NaN,8,E-76 approved on E-76 approved on,48.00,14.00,7.00,SCAG015,At 117 Signalized Intersections Throughout The...,Install Emergency Vehicle Pre-emption System.,4,NaT,NaT,NON-MPO,NaN,5441,046,False
11288,Obligated,RPSTPL,5955(077),Orange,2014-01-10,2014-07-14,2014-07-22,2014-07-22,2014-07-23,0.00,0.00,-615396.00,Authorized,NaN,12,E-76 approved on E-76 approved on,185.00,8.00,1.00,ORA990907,17th Street From Prospect Avenue North To Newp...,Landscape New Median And Pavement Rehabilitation,2,NaT,NaT,MCAG,NaN,5955,077,False
12282,Obligated,BHLS,5006(205),Los Angeles,2014-03-19,2014-03-20,2014-03-24,2014-04-04,2014-04-06,283134.00,0.00,353917.00,Authorized,NaN,7,E-76 approved on E-76 approved on,1.00,15.00,2.00,LA000800,"Riverside Dr. Over L.a. River, Bridge # 53c1298",Bridge Rehabilitation,6,NaT,NaT,SANDAG,NaN,5006,205,False
8549,Obligated,CML,5412(017),Rancho Mirage,2015-05-05,2015-08-07,2015-08-10,2015-08-10,2015-08-10,101000.00,0.00,114085.00,Authorized,NaN,8,E-76 approved on E-76 approved on,94.00,3.00,0.00,RIV140814,Bob Hope Dr.: Dinah Shore Dr. To Agua Caliente...,Install Four-foot High Sand Fencing,1,NaT,NaT,SCAG,NaN,5412,017,False
11214,Obligated,SRTSL,5207(034),La Mesa,2014-07-03,2014-07-24,2014-07-26,2014-07-26,2014-07-29,288400.00,0.00,391320.00,Authorized,NaN,11,E-76 approved on E-76 approved on,21.00,2.00,3.00,CAL 104 / CAL,In La Mesa At Various Locations,Construct Sidewalk And Other Improvements,2,NaT,NaT,SJCOG,NaN,5207,034,False
8054,Obligated,ESPLHP,5282(034),Palm Springs,2015-10-07,2015-10-07,2015-11-04,2015-11-04,2015-11-08,0.00,0.00,-9782001.35,Authorized,NaN,8,E-76 approved on E-76 approved on,0.00,28.00,4.00,NaN,I-10 @ Indian Cyn Dr. From 20th Ave. To Garne,I/c Modification,2,NaT,NaT,COFCG,NaN,5282,034,False
11838,Obligated,BRLS,5929(166),San Joaquin,2014-05-14,2014-05-15,2014-05-21,2014-05-28,2014-05-30,400000.00,0.00,500000.00,Authorized,NaN,10,E-76 approved on E-76 approved on,1.00,13.00,2.00,NaN,Mchenry Avenue Over Stanislaus River (bridge 3...,Bridge Replacement,3,NaT,NaT,NON-MPO,NaN,5929,166,False
12007,Obligated,BPMP,5940(111),Mariposa,2014-04-10,2014-04-11,2014-04-22,2014-04-28,2014-04-30,60000.00,0.00,60000.00,Authorized,NaN,10,E-76 approved on E-76 approved on,1.00,17.00,2.00,HBP-STWD,Multiple Locations (county Bridges),Preventive Maintenance (tc),1,NaT,NaT,STANCOG,NaN,5940,111,False


In [65]:
print(len(nan_rows))

1499


In [66]:
#checking to see if these NaN values are also in the downloadable data. 

In [67]:
# There are the same number of NaNs (1499) on the excel sheet. 
# So without these, there are still 2,206 

## Comparing columns and Summarizing Data

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20445 entries, 0 to 20446
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   location                20445 non-null  object        
 1   prefix                  20445 non-null  object        
 2   project_no              20445 non-null  object        
 3   agency                  20445 non-null  object        
 4   prepared_date           20427 non-null  datetime64[ns]
 5   submit__to_hq_date      20270 non-null  datetime64[ns]
 6   hq_review_date          20238 non-null  datetime64[ns]
 7   submit_to_fhwa_date     20220 non-null  datetime64[ns]
 8   to_fmis_date            20220 non-null  datetime64[ns]
 9   fed_requested           20444 non-null  float64       
 10  ac_requested            20445 non-null  float64       
 11  total_requested         20444 non-null  float64       
 12  status_comment          20445 non-null  object

In [69]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
6991,Obligated,CML,5107(028),Santa Monica,2016-04-13,2016-04-13,2016-04-13,2016-04-13,2016-04-25,-19.67,0.00,7507.32,Authorized,5107,7,E-76 approved on Prepared on 10/12/2015,0.00,0.00,12.00,LAF1728,Const. Real Time Parking For Motorist Tarvelin...,Real-time Beach Parking Signs,3,NaT,NaT,SCAG,NaN,5107,028,True
799,Obligated,HSIPL,5138(049),Santa Maria,2018-07-30,2018-07-30,2018-07-30,2018-07-31,2018-08-14,0.00,0.00,-208569.62,Authorized,5138,5,E-76 approved on,NaN,1.00,14.00,LOCALHSIP,In The City Of Santa Maria On N Depot W Main T...,Modify Traffic Signal & Other Safety Improvem...,4,NaT,NaT,SBCG,NaN,5138,049,True
9731,Obligated,BRLS,5005(089),San Jose,2015-03-05,2015-03-05,2015-03-05,2015-03-10,2015-03-10,-20053.60,0.00,0.00,Authorized,5005,4,E-76 approved on E-76 approved on,0.00,5.00,0.00,NaN,Santa Clara St Over Coyote Creek (br#37c0033),Bridge Replacement,2,NaT,NaT,MTC,NaN,5005,089,True
11770,Obligated,CML,6272(033),San Francisco County Transportation Authority,2014-05-14,2014-05-14,2014-05-27,2014-06-06,2014-06-11,0.00,0.00,0.00,Authorized,6272,4,E-76 approved on E-76 approved on,0.00,23.00,5.00,REG110012,Various Locations In San Francisco And Berkeley,Tdm Strategy Expansion&evaluation; Outreach,3,NaT,NaT,MTC,NaN,6272,033,True
2674,Obligated,BRLO,5911(056),Glenn County,2018-01-14,2018-01-14,2018-01-18,2018-01-19,2018-01-25,265000.00,0.00,265000.00,Authorized,5911,3,E-76 approved on,12.00,5.00,6.00,HBPLOCAL,C.r. 305 At Watson Creek ~0.1 Mile East Of C.r...,Bridge Replacement (tc),2,2018-01-02,2018-01-02,NON-MPO,NaN,5911,056,True
7042,Obligated,BHLO,5006(221),Los Angeles,2016-02-08,2016-03-10,2016-03-23,2016-03-23,2016-04-13,560564.00,0.00,633191.00,Authorized,5006,7,E-76 approved on Prepared on 10/12/2015,31.00,13.00,21.00,LA0C8063,"Riverside Dr. Over L.a. River, Brdg #53c0160",Bridge Replacement /widening,9,NaT,NaT,SCAG,NaN,5006,221,True
12168,Obligated,STPL,5109(192),Bakersfield,2014-03-24,2014-03-26,2014-03-27,2014-04-14,2014-04-14,0.00,0.00,1273.95,Authorized,5109,6,E-76 approved on E-76 approved on,2.00,19.00,0.00,KER120402,Martin Luther King Blvd From Wilkins Street To...,Pavement Rehabilitation,2,NaT,NaT,KCOG,NaN,5109,192,True
5215,Obligated,STPL,5317(015),Newark,2016-12-20,2016-12-22,2016-12-22,2016-12-23,2016-12-23,454000.00,0.00,734328.00,Authorized,5317,4,E-76 approved on Prepared on 10/6/2015,2.00,1.00,0.00,ALA130027,Enterprise Drive- Filbert Street To Approximat...,"Implement Road Diet And Rehabilitate Pavement,...",1,NaT,NaT,MTC,NaN,5317,015,True
16313,Obligated,ER,40A0(052),Humboldt County,2020-02-24,2020-02-24,2020-03-29,2020-04-03,2020-04-06,56659.20,0.00,64000.00,Authorized,5904,1,E-76 approved on,3.00,39.00,3.00,NaN,Alderpoint Road Pm 42.00,Permanent Restoration,1,2020-02-21,2020-02-24,NON-MPO,NaN,40A0,052,False
8774,Obligated,BRLO,5914(068),Lake County,2015-02-25,2015-07-09,2015-07-10,2015-07-10,2015-07-10,60000.00,0.00,60000.00,Authorized,5914,1,E-76 approved on E-76 approved on,134.00,1.00,0.00,HBPLOCAL,Foard Road Bridge Over Anderson Creek (bridge ...,Replace Existing 1 Lane Bridge (tc),4,NaT,NaT,NON-MPO,NaN,5914,068,True


In [70]:
from siuba.data import cars_sql
from siuba import group_by, mutate, show_query

In [71]:
df[_.compare_id_locode == False]

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.00,13000.00,Authorized,5910,1,E-76 approved on,1.00,1.00,7.00,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,008,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.00,51394.58,Authorized,5904,1,E-76 approved on,14.00,0.00,6.00,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,004,False
21,Obligated,ACSTP,32L0(510),Butte County,2018-12-17,2018-12-20,2018-12-20,2018-12-20,2018-12-21,0.00,2986207.00,3367920.00,Authorized,5912,3,E-76 approved on,43.00,0.00,1.00,NaN,Oroville Quincy Hwy: Location #1 100 Yards Ab...,Emergency Opening And Restoration Of Roadway S...,1,2018-11-07,2018-11-07,BCAG,NaN,32L0,510,False
24,Obligated,ER,26E0(001),Napa County,2018-12-18,2018-12-18,2018-12-20,2018-12-20,2018-12-21,941474.00,0.00,1026469.00,Authorized,5921,4,E-76 approved on,13.00,2.00,1.00,NaN,Daf 1: Oak Knoll Ave (mpm 0.20); Daf 2: Oak Kn...,Napa County Earthquake Repair - Pavement Resto...,4,2018-11-30,2018-12-05,MTC,NaN,26.00,001,False
54,Obligated,ER,29P0(001),Carlsbad,2018-12-13,2018-12-13,2018-12-13,2018-12-14,2018-12-18,-61058.59,0.00,-53619.67,Authorized,5308,11,E-76 approved on,NaN,1.00,4.00,NaN,"Carlsbad Boulevard, North Of Las Encinas Creek",Emergency Revetment,2,NaT,NaT,SDAG,NaN,29P0,001,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20337,Obligated,ER,32L0(440),Monterey County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-22,-232125.67,0.00,-255334.09,Authorized,5944,5,E-76 approved on,4.00,0.00,4.00,NaN,Hall Road At Pini Road,Eo- County Placed Barricades. Contractor Plac...,6,2021-10-14,2021-10-15,AMBAG,NaN,32L0,440,False
20361,Obligated,ER,32L0(560),Los Angeles County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-26,-5796.69,0.00,-6547.71,Authorized,5953,7,E-76 approved on,4.00,0.00,8.00,NaN,"The Old Road @ Mm12.62, 13.66 And @ Weldon Rd;...","Eo: Removed Debris Along The Roadway, Shoulder...",2,2021-10-14,2021-10-15,SCAG,NaN,32L0,560,False
20366,Obligated,ER,30X0(001),Los Angeles County,2021-10-18,2021-10-18,2021-10-18,2021-10-18,2021-10-22,0.00,0.00,-195.52,Authorized,5953,7,E-76 approved on,4.00,0.00,4.00,NaN,Soledad Canyon Rd From Mm 15.48 To Mm 18.46,Drainage Structures Installation,3,2021-10-14,2021-10-15,SCAG,NaN,30X0,001,False
20369,Obligated,ER,32L0(554),Trinity County,2021-10-13,2021-10-13,2021-10-13,2021-10-13,2021-10-19,-243.30,0.00,-274.82,Authorized,5953,7,E-76 approved on,6.00,0.00,6.00,NaN,Kanan Dume Rd From Mulholland Hwy To City Mal...,Eo: Debris Removal,3,2021-10-07,2021-10-08,SCAG,NaN,32L0,554,False


In [72]:
df['status_comment'].value_counts()

Authorized                                                         19642
..                                                                   194
Prog Code Z400                                                       147
Prog Code M400                                                        73
Prog Code Z230                                                        60
                                                                   ...  
ON HOLD: INSUFFICIENT SACOG CMAQ APPORTIONMENT BALANCE                 1
HOLD - REQ FOR OBLIGATION OF FLEX STBGP FROM MTC LOAN TO BRIDGE        1
MISSING REASON FOR THE DELAY                                           1
EXPIRED QAP                                                            1
prog code Z300                                                         1
Name: status_comment, Length: 81, dtype: int64

In [73]:
df['location'].value_counts()

Obligated          19642
FTA Transferred      543
DISTRICT             170
HQ                    53
FMIS                  37
Name: location, dtype: int64

In [74]:
df['type_of_work'].nunique()

6882

## Condensing Prefix Variables

In [80]:
df['prefix'].nunique()

392

In [82]:
df['prefix'].value_counts().sort_index(ascending=True)

6140000      2
ACNHPI       2
ACST-ER    238
ACSTP      317
ASCTP        1
          ... 
VPPL        19
VPPP         3
VPPPL        5
VPPTCS       1
WZDXL20      1
Name: prefix, Length: 392, dtype: int64

In [83]:
df.prefix.replace(['BPMP'], ['BPMPL'], inplace=True)
df.prefix.replace(['BR-'], ['BR'], inplace=True)

df.prefix.replace(['VPPPL'], ['VPPP'], inplace=True)

df.prefix.replace(['ATP-CML'], ['ATPCML'], inplace=True)
df.prefix.replace(['ATPL'], ['ATP'], inplace=True)

df.prefix.replace(['CASB003'], ['CASB'], inplace=True)
df.prefix.replace(['CASB05'], ['CASB'], inplace=True)
df.prefix.replace(['CASB06'], ['CASB'], inplace=True)
df.prefix.replace(['CASB07'], ['CASB'], inplace=True)
df.prefix.replace(['CASB09'], ['CASB'], inplace=True)
df.prefix.replace(['CASB10'], ['CASB'], inplace=True)
df.prefix.replace(['CASB11'], ['CASB'], inplace=True)
df.prefix.replace(['CASB12'], ['CASB'], inplace=True)
df.prefix.replace(['CASB803'], ['CASB'], inplace=True)
df.prefix.replace(['CASB902'], ['CASB'], inplace=True)
df.prefix.replace(['CASB904'], ['CASB'], inplace=True)
df.prefix.replace(['CASB905'], ['CASB'], inplace=True)
df.prefix.replace(['CASB12'], ['CASB'], inplace=True)
df.prefix.replace(['CASBL'], ['CASB'], inplace=True)

df.prefix.replace(['DEM03L'], ['DEM'], inplace=True)

df.prefix.replace(['FERPL16'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL17'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL18'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPL19'], ['FERPL'], inplace=True)
df.prefix.replace(['FERPLN16'], ['FERPL'], inplace=True)

df.prefix.replace(['FSP11'], ['FSP'], inplace=True)
df.prefix.replace(['FSP12'], ['FSP'], inplace=True)
df.prefix.replace(['FSP13'], ['FSP'], inplace=True)
df.prefix.replace(['FSP14'], ['FSP'], inplace=True)

df.prefix.replace(['HRRRL'], ['HRRR'], inplace=True)
df.prefix.replace(['IMDL'], ['IMD'], inplace=True)
df.prefix.replace(['HSIPSTPL'], ['HSIPSTP'], inplace=True)
df.prefix.replace(['PNRSTCIL'], ['PNRSTCI'], inplace=True)
df.prefix.replace(['RSTPL'], ['RSTP'], inplace=True)
df.prefix.replace(['RPSTPLE-'], ['RPSTPLE'], inplace=True)
df.prefix.replace(['TILUL'], ['TILU'], inplace=True)
df.prefix.replace(['STPL'], ['STP'], inplace=True)
df.prefix.replace(['RPSTPLE-'], ['RPSTPLE'], inplace=True)


df.prefix.replace(['HSIPL'], ['HSIP'], inplace=True)
df.prefix.replace(['HSIP5'], ['HSIP'], inplace=True)
df.prefix.replace(['HSIIPL'], ['HSIP'], inplace=True)
df.prefix.replace(['HISP5'], ['HSIP'], inplace=True)

df.prefix.replace(['NCPDL0'], ['NCPD'], inplace=True)
df.prefix.replace(['NCPD02'], ['NCPD'], inplace=True)

df.prefix.replace(['FERPLN16'], ['FERPL'], inplace=True)

df.prefix.replace(['PLHL04'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL05'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL10'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL11'], ['PLHL'], inplace=True)

df.prefix.replace(['PLHDL05'], ['PLDHL'], inplace=True)
df.prefix.replace(['PLHDL06'], ['PLDHL'], inplace=True)
df.prefix.replace(['PLHDL08'], ['PLDHL'], inplace=True)
df.prefix.replace(['PLHL04'], ['PLHL'], inplace=True)
df.prefix.replace(['PLHL05'], ['PLHL'], inplace=True)

df.prefix.replace(['RPSTPL'], ['RPSTP'], inplace=True)
df.prefix.replace(['RPSTPL-'], ['RPSTP'], inplace=True)

df.prefix.replace(['SRTSL'], ['SRTS'], inplace=True)
df.prefix.replace(['SRTSL-'], ['SRTS'], inplace=True)

df.prefix.replace(['TGR2DGL'], ['TGR2DG'], inplace=True)
df.prefix.replace(['TGR2DG.'], ['TGR2DG'], inplace=True)

df.prefix.replace(['TCSPL'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP02'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP03'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP05'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP06'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP08'], ['TCSP'], inplace=True)
df.prefix.replace(['TCSP10'], ['TCSP'], inplace=True)

In [84]:
df['prefix'].nunique()

335

In [86]:
print(sorted(df.prefix.unique()))

['6140000', 'ACNHPI', 'ACST-ER', 'ACSTP', 'ASCTP', 'ATCMTD', 'ATF5512', 'ATP', 'ATPCML', 'ATPHIPL', 'ATPHSIP', 'ATPLN', 'ATPLNI', 'ATPLSB1', 'ATPNIL', 'ATPSB1', 'ATPSB1L', 'ATPSTC', 'ATPSTCML', 'ATPSTPL', 'BDGL', 'BGFL', 'BHL', 'BHLO', 'BHLOZ', 'BHLOZD', 'BHLS', 'BHLSCR', 'BHLSP', 'BHLSZ', 'BHLSZD', 'BPMPL', 'BR', 'BR-NBISL', 'BRCMLO', 'BRHIPL', 'BRIM', 'BRL', 'BRLKS', 'BRLNS', 'BRLO', 'BRLOZ', 'BRLOZA', 'BRLOZB', 'BRLOZE', 'BRLRT', 'BRLS', 'BRLSN', 'BRLSZ', 'BRLSZA', 'BRLSZD', 'BRLZ', 'BRNBIF', 'BRNBISL', 'BROS', 'BUILDL', 'CASB', 'CFLBRLO', 'CMAQL', 'CMESPL', 'CMFERP', 'CMFERP16', 'CMHIPL', 'CMHPLU', 'CMHPLULN', 'CMHRL', 'CMHSIPL', 'CML', 'CMLFSP', 'CMLFSP18', 'CMLFTA', 'CMLG', 'CMLGNI', 'CMLHPL', 'CMLN', 'CMLNGI', 'CMLNI', 'CMLSBCG', 'CMLSTBGL', 'CMRPSTPL', 'CMSTPL', 'CMSTPLN', 'CMSTPLNI', 'CMTCIFL', 'CMTCIFLN', 'CMTLSP', 'CRDL', 'CRP', 'CRRSAA', 'DBF', 'DBPL02', 'DE', 'DEM', 'DEM04L', 'DEM05', 'DEM05L', 'DEM05LN', 'DEM05U', 'DEM05UBL', 'DEM06', 'DEM06L', 'DEM06U', 'DEM06UBL', 'DEM0

In [88]:
prefix = df['prefix'].value_counts()

In [89]:
prefix.head()

HSIP    3367
CML     3334
STP     2817
ER      1607
BRLO    1416
Name: prefix, dtype: int64

In [90]:
print((prefix == 1).sum())

87


In [91]:
prefix.sort_index(inplace=True)
prefix

6140000       2
ACNHPI        2
ACST-ER     238
ACSTP       317
ASCTP         1
           ... 
USFSSTPL      4
VPPL         19
VPPP          8
VPPTCS        1
WZDXL20       1
Name: prefix, Length: 335, dtype: int64

In [86]:
#Getting a CSV to create a detailed crosswalk with defintions. 
#prefix.to_csv('count_grant_types.csv')

## Convert to CSV for future analysis 

In [93]:
#df.to_csv('clean_obligated_waiting.csv')